# Basics

- bson = binary json

- `mongosh` creates a MongoDB shell that is in JavaScript

- You must start a server before connecting to it in VSCode

- run `mongod` to start server

## find command

`cooker> db.<collection_name>.find()`

- creates a new collection if it doesn't exist
- find() returns everything inside that collection

### conditionals:

`cooker> db.recipes.find({title: "Tacos", title: 1
})`

Returns all items with title Tacos. Returns only their titles and ids.

`cooker> db.recipes.find({title: "Tacos", title: 0
})`

Returns everything but the titles of items with title Tacos

`cooker> db.recipes.find({},
{title: 1
})`

Returns all items' titles

#### Regex

because we are running a JavaScript shell in MongoDB we can use regex.

`db.recipes.find({title: { $regex: /taco/i
  }
})`

It is looking for documents where the "title" field matches a regular expression. The regular expression used is /taco/i, where "taco" is the pattern we are searching for and the i flag makes the search case-insensitive.